In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 25s 6us/step


In [4]:
train_images=train_images/255.0
test_images=test_images/255.0
#

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#hp.int will create the renge of integers with conv_1_filter name where minimum value is 64 which filter will start from 32 
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-8-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 03m 00s]
val_accuracy: 0.9006666541099548

Best val_accuracy So Far: 0.9023333191871643
Total elapsed time: 00h 22m 00s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        153664    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2867312   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,024,602
Trainable params: 3,024,602
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 156s 92ms/step - loss: 0.1648 - accuracy: 0.9385 - val_loss: 0.2718 - val_accuracy: 0.9085
Epoch 5/10
1688/1688 [==============================] - 154s 91ms/step - loss: 0.1278 - accuracy: 0.9520 - val_loss: 0.3045 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 155s 92ms/step - loss: 0.0932 - accuracy: 0.9650 - val_loss: 0.3009 - val_accuracy: 0.9083
Epoch 7/10
1688/1688 [==============================] - 157s 93ms/step - loss: 0.0748 - accuracy: 0.9716 - val_loss: 0.3724 - val_accuracy: 0.9073
Epoch 8/10
1688/1688 [==============================] - 155s 92ms/step - loss: 0.0529 - accuracy: 0.9808 - val_loss: 0.4026 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 157s 93ms/step - loss: 0.0460 - accuracy: 0.9839 - val_loss: 0.4707 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 160s 95ms/step - loss: 0.0364 - accuracy: 0.9871 - val_loss: 